In [18]:
import pandas as pd
import numpy as np
from os import walk
from tqdm import tqdm
import datetime

In [19]:
files = [i[2] for i in walk('/Users/skostuchik/projects/robert/files')][0]
raw_data = pd.DataFrame()

In [20]:
def labels_formatting(metric):
    '''Форматирование названий показателей - удаление пробелов в начале и замена на _'''
    cnt = 0
    for i in metric:
        if i == ' ':
            cnt += 1
        else:
            return(metric[cnt:].replace(' ', '_').replace(':', ''))

In [55]:
for i in tqdm(files):
    df = pd.read_csv(f'/Users/skostuchik/projects/robert/files/{i}')
    df = df.loc[24:]
    df = df[df['0'].notna()]
    df['0'] = df['0'].apply(labels_formatting)
    df = df.T
    
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    
    df['company'] = i.split('_')[2]
    
    raw_data = raw_data.append(df)

100%|██████████| 11/11 [00:00<00:00, 27.02it/s]


In [22]:
raw_data = raw_data.replace('-', np.nan)
raw_data = raw_data.replace('TTM/current', np.nan)

In [23]:
raw_data = raw_data.loc[:,~raw_data.columns.duplicated()]

In [24]:
str_cols = ['company']
date_cols = ['Fiscal_Period', 'Restated_Filing_Date', 'Filing_Date', 'Earnings_Release_Date']

In [25]:
def float_correction(x):
    x = str(x).replace(',','')
    
    if ' M' in x:
        x = float(x.replace(' M','')) * 1000000
    else:
        x = float(x)
    
    return(x)

In [26]:
for i in tqdm(raw_data.columns):
    if i in date_cols:
        if i == 'Fiscal_Period':
            #raw_data[i] = pd.to_datetime(raw_data['Fiscal_Period'].apply(lambda x: str(x)+'-01' if pd.notna(x) else np.nan))
            continue
        else:
            raw_data[i] = pd.to_datetime(raw_data[i])
        
    elif i in str_cols:
        continue
    else:
        #print(i)
        raw_data[i] = raw_data[i].apply(float_correction)

100%|██████████| 226/226 [00:00<00:00, 295.37it/s]


In [27]:
print(date_from_1, date_to_1)

10/21/19 11/20/19


In [28]:
raw_data[(raw_data['Filing_Date'] >= date_from_1)&(raw_data['Filing_Date'] <= date_to_1)]['Filing_Date'].unique()

array(['2019-11-20T00:00:00.000000000', '2019-10-22T00:00:00.000000000',
       '2019-10-29T00:00:00.000000000', '2019-11-19T00:00:00.000000000',
       '2019-10-31T00:00:00.000000000', '2019-10-23T00:00:00.000000000',
       '2019-10-25T00:00:00.000000000', '2019-11-13T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000'], dtype='datetime64[ns]')

# Case 1

In [29]:
raw_data[['Filing_Date','Month_End_Stock_Price']]

0,Filing_Date,Month_End_Stock_Price
1,2020-11-25,NaN
2,2020-11-25,124.08
3,2019-11-20,130.32
4,2018-11-21,116.94
5,2017-11-22,98.57
...,...,...
75,2007-04-09,39.25
76,2007-02-05,40.16
77,2006-10-11,32.44
78,2006-07-11,28.63


In [30]:
date_to_1 = '11/20/19'
date_from_1 = (datetime.datetime(2019, 11, 20) - datetime.timedelta(days=30)).strftime("%m/%d/%y")

In [32]:
#указать формат даты!!!
df_1 = raw_data[(raw_data['Filing_Date'] >= date_from_1)&(raw_data['Filing_Date'] <= date_to_1)]

In [38]:
index_first_price = df_1[df_1['PE_Ratio']<20]['Month_End_Stock_Price'].sum()
companies_ticker = df_1[df_1['PE_Ratio']<20]['company'].unique().tolist()

In [39]:
date_from_2 = '09/20/20'
date_to_2 = '11/20/20'

In [48]:
df_2 = raw_data[(raw_data['Filing_Date'] >= date_from_2)&(raw_data['Filing_Date'] <= date_to_2)&
                (raw_data['company'].isin(companies_ticker))]

In [50]:
index_second_price = df_2['Month_End_Stock_Price'].sum()

In [51]:
print(f'''
First Price: {index_first_price}, 
Second Price: {index_second_price},
% Change Price: {index_second_price / index_first_price - 1}
''')


First Price: 260.64, 
Second Price: 429.01,
% Change Price: 0.6459868017188459



***

In [ ]:
c_names = raw_data.columns.tolist()

In [ ]:
c_names.sort()

In [ ]:
c_names